# Cthulhu.jl Workshop

Shuhei Kadowaki

<h2>Workshop Outline<span class="tocSkip"></span></h2>
<div class="toc"><ul class="toc-item"><li><span><a href="#Overview-of-Cthulhu.jl" data-toc-modified-id="Overview-of-Cthulhu.jl-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Overview of Cthulhu.jl</a></span><ul class="toc-item"><li><span><a href="#Walk-through-static-call-graph" data-toc-modified-id="Walk-through-static-call-graph-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Walk-through static call graph</a></span></li><li><span><a href="#Walk-through-program-representations" data-toc-modified-id="Walk-through-program-representations-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Walk-through program representations</a></span></li></ul></li><li><span><a href="#Get-familiar-with-Julia-compiler-with-Cthulhu" data-toc-modified-id="Get-familiar-with-Julia-compiler-with-Cthulhu-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Get familiar with Julia compiler with Cthulhu</a></span><ul class="toc-item"><li><span><a href="#Dynamic/static-dispatch" data-toc-modified-id="Dynamic/static-dispatch-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Dynamic/static dispatch</a></span></li><li><span><a href="#Multiple-matching-methods-and-union-split" data-toc-modified-id="Multiple-matching-methods-and-union-split-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Multiple matching methods and union-split</a></span></li><li><span><a href="#Multithreading-code" data-toc-modified-id="Multithreading-code-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Multithreading code</a></span></li></ul></li></ul></div>

In [1]:
using BenchmarkTools, Cthulhu, JETTest

## Overview of Cthulhu.jl

### Walk-through static call graph

Cthulhu is a lens into a static call graph, and it's very similar to `code_typed`. 
They _never_ execute an input program as opposed to ordinal debuggers that interact with runtime.

To illustrate how Cthulhu is different from `code_typed`, consider the following example

In [2]:
function inner(T, somes)
    r::T = zero(T)
    for s in somes
        if isa(s.value, T)
            r += s.value
        end
    end
    return r
end
outer(somes) = inner(Int, somes)

somes = [Some(rand(Bool) ? nothing : i) for i in 1:100000]
@benchmark outer($somes)

BenchmarkTools.Trial: 458 samples with 1 evaluation.
 Range (min … max):   9.241 ms …  15.169 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     10.915 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   10.907 ms ± 885.843 μs  ┊ GC (mean ± σ):  0.43% ± 1.70%

                       ▆▅█▆▄▁▄▃          ▂                      
  ▄▄▄▄▇█▆█▆▆▇▆▆█▄▅▄▄▃▅█████████▇▆▆█▆▆█▆▆▄█▄▄▃▄▄▂▃▁▁▁▃▃▁▂▂▂▁▁▁▂ ▄
  9.24 ms         Histogram: frequency by time         13.4 ms <

 Memory estimate: 2.28 MiB, allocs estimate: 149332.

Suppose we want to improve the performance of `outer`. 

As [the performance tips suggest](https://docs.julialang.org/en/v1/manual/performance-tips/#tools), we first take a look at `@code_warntype`:

In [3]:
@code_warntype outer(somes)

MethodInstance for outer(::Vector{Some})
  from outer(somes) in Main at In[2]:10
Arguments
  #self#::Core.Const(outer)
  somes::Vector{Some}
Body::Int64
1 ─ %1 = Main.inner(Main.Int, somes)::Int64
└──      return %1



Looks all good ?

Now we give a try to [JETTest.jl](https://github.com/aviatesk/JETTest.jl):

In [4]:
@report_dispatch outer(somes)

═════ 2 possible errors found ═════
┌ @ In[2]:10 Main.inner(Main.Int, somes)
│┌ @ In[2]:5 Main.+(%22, %26)
││ runtime dispatch detected: Main.+(%22::Int64, %26::Any)
│└───────────
│┌ @ In[2]:5 Base.convert(T, %27)
││ runtime dispatch detected: Base.convert(T::Type{Int64}, %27::Any)
│└───────────


(Int64, 2)

There _are_ type-instabilities ! What's wrong with `code_warntype` ?

Now let's use Cthulhu and see what really happened:
```julia
@descend_code_warntype optimize=false outer(somes)
```

_Go to terminal_

In [5]:
function inner(T, somes)
    r::T = zero(T)
    for s in somes
        val = s.value
        if isa(val, T)
            r += val::T
        end
    end
    return r
end

@benchmark outer($somes) # 3x faster !

BenchmarkTools.Trial: 1844 samples with 1 evaluation.
 Range (min … max):  2.158 ms …   6.349 ms  ┊ GC (min … max): 0.00% … 52.38%
 Time  (median):     2.593 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.690 ms ± 406.191 μs  ┊ GC (mean ± σ):  1.21% ±  5.36%

       ▅██▆▆█▇▅▄▁                                              
  ▃▂▂▄▆████████████▇▇▆▅▅▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▃▂▁▂▂▁▂▂▂▂▁▂▂ ▄
  2.16 ms         Histogram: frequency by time        4.45 ms <

 Memory estimate: 776.64 KiB, allocs estimate: 49705.

Yay, now it runs 3x faster !

(For expert: <https://github.com/JuliaLang/julia/pull/41199> will eliminate the need for this kind of variable extraction in the future :))

Recap:
- `code_typed` and its family only presents the final output of inference and optimization
- with Cthulhu, we can walk-through a static call graph by descending into each intermediate state
- Cthulhu doesn't suggest anything, _it's up to you how to interpret them_
  - in this view, we can think of JET as, an automated and opinionated version of a workflow using Cthulhu

### Walk-through program representations

Julia's compilation process consists of:
1. _lowering_: AST -> lowered code
2. _type inference_: lowered code -> unoptimized typed code
3. _Julia-level optimization_: unoptimized typed code -> optimized typed code
4. _LLVM optimization_: optimized typed code -> LLVM IR
5. _LLVM code generation_: LLVM IR -> native code

So there are various different program representations:
- AST (`Core.Expr`)
- lowered code (`Core.CodeInfo`)
- unoptimized typed code (`Core.CodeInfo`)
- optimized typed code (`Core.CodeInfo`, `Core.Compiler.IRCode`)
- LLVM IR
- native code

Cthulhu lets us to traverse these forms quite easily (except AST) !
- unoptimized typed code: to see how type inference goes
- optimized typed code: to see Julia-level optimizations
- LLVM IR: to enjoy all the tricks LLVM does for us
- native code: if you doubt all the beauty of abstractions

As an example, let's take a deeper look into this function call.

```julia
descend(; optimize=false) do
    ary = [1,2,3]
    @inbounds ary[1] + ary[3]
end
```

Note that:
- inference annotates types for this function call
- Julia-level optimization inlines typed function calls
- LLVM further gets rid of the array indexing and folds the final output (still it's not smart enough to eliminate the allocation of `ary`)

_Go to terminal_

## Get familiar with Julia compiler with Cthulhu

We need some knowledge of and familiarity with some concepts of Julia compilation to understand what Cthulhu shows us.

Here I'm going to show a selection of most important concepts with examples.

### Dynamic/static dispatch

When looking at an optimized code, there are two different representation of function calls: `:invoke` and  `:call`.
Here are their meanings:
- `:invoke` is a statically-resolved generic function call (i.e. "static dispatch")
- `:call` is a dynamic generic function call (i.e. "dynamic dispatch"), or a builtin function call

Note that it depends on how type inference goes successfully if Julia compiler can resolve function calls at compiletime or just delays the resolution to runtime.

```julia
@noinline invokecall(a) = sum(sincos(a))
v = 42.0 # type-inference barrier here !

# how these `f` calls are resolved
descend((Int,)) do a
    good = invokecall(a)
    bad  = invokecall(v)
    return (good, bad)
end
```

_Go to terminal_

And now we just understood the idea of how JETTest works – it just hunts for `:call`s appearing in optimized typed code.

### Multiple matching methods and union-split

When there are multiple matching methods, Julia's type inference recurses into each method match, so does Cthulhu (caveat: not always, think of e.g. what can happen for `show(::IO, ::Any)`).

```julia
multiplem(::Int)     = :int
multiplem(::Integer) = "integer"

descend((Integer,); optimize=false) do i
    multiplem(i)
end
```

_Go to terminal_

Note that the return type was `Union{String,Symbol}`.

`Union` is basically "a enumeration of possible types".
Julia compiler will form `Union` when it can't narrow down the type into one.

Julia compiler will "split" `Union` type when a call signature includes `Union`, and this often lead to multiple matching methods:
```
descend((Integer,); optimize=false) do i
    r = multiplem(i)
    string(r)
end
```

_Go to terminal_

More interesting thing happens when looking at optimized code.

```julia
verytyped(a::Int) = sin_for_int(a)
verytyped(a::Float64) = sin_for_general(a)

# see unoptimized code, then look at optimized code
descend((Any,); optimize=false) do a
    verytyped(a)
end
```
_Go to terminal_

Julia automatically forms `isa` branches and it _expands_ the union-split, so that the `verytyped` call can be resolved statically.

Excersice:
```julia
verytyped2(a::Int) = sin_for_int(a)
verytyped2(a) = sin_for_general(a)

# see unoptimized code, then look at optimized code
descend((Any,); optimize=false) do a
    verytyped2(a)
end
```

(it might be interesting if you consider this example together with the fact that our manual says ["In general, you should use the most general applicable abstract types for arguments, and when in doubt, omit the argument types."]("https://docs.julialang.org/en/v1.8-dev/manual/functions/#Argument-type-declarations"))

### Multithreading code

Reasoning about multithreading code is hard even with Cthulhu (this implies that Julia compiler is also having hard time inferring and optimizing multithreading code).

Let's consider optimizing this code:

In [6]:
function compute_sins(i)
    n = 1000000
    out = Vector{Float64}(undef, n)
    Threads.@threads for itr in 1:n
        i = itr
        out[i] = sin(i)
    end
    return out
end

@benchmark compute_sins(1000000)

BenchmarkTools.Trial: 111 samples with 1 evaluation.
 Range (min … max):  39.952 ms … 56.223 ms  ┊ GC (min … max): 0.00% … 4.94%
 Time  (median):     44.851 ms              ┊ GC (median):    4.36%
 Time  (mean ± σ):   45.349 ms ±  3.016 ms  ┊ GC (mean ± σ):  3.98% ± 1.74%

           █▂  ▂▂▅▂      ▃          ▂                          
  ▅▄▅▁▄▅▄▅████▁████▇▅▄▇▅▅███▅▅▇▇▅▅▇▄█▁▄▄▄▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄ ▄
  40 ms           Histogram: frequency by time        56.2 ms <

 Memory estimate: 38.14 MiB, allocs estimate: 1999499.

In [7]:
@code_warntype compute_sins(1000000)

MethodInstance for compute_sins(::Int64)
  from compute_sins(i) in Main at In[6]:1
Arguments
  #self#::Core.Const(compute_sins)
  i@_2::Int64
Locals
  threadsfor_fun::var"#208#threadsfor_fun#5"{Vector{Float64}, UnitRange{Int64}}
  out::Vector{Float64}
  n::Int64
  range::UnitRange{Int64}
  i@_7::Union{Int64, Core.Box}
Body::Vector{Float64}
1 ─       (i@_7 = i@_2)
│         (i@_7 = Core.Box(i@_7::Int64))
│         (n = 1000000)
│   %4  = Core.apply_type(Main.Vector, Main.Float64)::Core.Const(Vector{Float64})
│         (out = (%4)(Main.undef, n::Core.Const(1000000)))
│   %6  = (1:n::Core.Const(1000000))::Core.Const(1:1000000)
│         (range = %6)
│   %8  = Main.:(var"#208#threadsfor_fun#5")::Core.Const(var"#208#threadsfor_fun#5")
│   %9  = Core.typeof(out)::Core.Const(Vector{Float64})
│   %10 = Core.typeof(range::Core.Const(1:1000000))::Core.Const(UnitRange{Int64})
│   %11 = Core.apply_type(%8, %9, %10)::Core.Const(var"#208#threadsfor_fun#5"{Vector{Float64}, UnitRange{Int64}})
│   %12 

There is `Core.Box`, which indicates something got captured by closure and it generally tends to be a source of bad performance. But it's not clear where it actually impacts the performance from the output of `@code_warntype`.

Seemingly a problem lives in a `Base.Threads.threading_run` call ? Let's run Cthulhu and descent into there:
```julia
@descend compute_sins(1000000)
```

_Go to terminal_

So the problem stemmed from the fact that the dead argument `i` is captured within a loop.
We can just eliminate it and now it runs 3x faster.

In [8]:
function compute_sins()
    n = 1000000
    out = Vector{Float64}(undef, n)
    Threads.@threads for itr in 1:n
        i = itr
        out[i] = sin(i)
    end
    return out
end

@benchmark compute_sins()

BenchmarkTools.Trial: 306 samples with 1 evaluation.
 Range (min … max):  14.424 ms … 24.539 ms  ┊ GC (min … max): 0.00% … 14.86%
 Time  (median):     15.564 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   16.343 ms ±  1.831 ms  ┊ GC (mean ± σ):  3.70% ±  7.04%

   ▄█▅▅▄▄             ▅ ▁                                      
  ▆██████▆▇▅▅▆▄▃▄▃▃▁▆██▄█▅▃▃▃▃▁▃▃▅▃▄▄▄▄▃▄▃▃▄▃▃▁▃▄▃▄▃▄▃▁▃▁▁▃▁▃ ▄
  14.4 ms         Histogram: frequency by time        21.4 ms <

 Memory estimate: 7.63 MiB, allocs estimate: 8.

In [9]:
# if you're interested, JETTest.jl can also tell the performance pitfall
# but with some other reports that stem from multithread constructions
@report_dispatch compute_sins(1000)

═════ 6 possible errors found ═════
┌ @ threadingconstructs.jl:97 Base.Threads.threading_run(threadsfor_fun)
│┌ @ threadingconstructs.jl:30 Base.Threads.Task(func)
││┌ @ threadingconstructs.jl:52 #self#(false)
│││┌ @ In[6]:6 Base.setindex!(%88, %87, %94)
││││ runtime dispatch detected: Base.setindex!(%88::Vector{Float64}, %87::Any, %94::Any)
│││└───────────
│││┌ @ In[6]:6 Main.sin(%86)
││││ runtime dispatch detected: Main.sin(%86::Any)
│││└───────────
│┌ @ threadingconstructs.jl:38 Base.Threads.wait(Base.getindex(tasks, i))
││┌ @ task.jl:320 Base._wait(t)
│││┌ @ task.jl:293 Base.wait(%30)
││││ runtime dispatch detected: Base.wait(%30::Any)
│││└───────────────
│││┌ @ task.jl:290 Base.lock(%13)
││││ runtime dispatch detected: Base.lock(%13::Any)
│││└───────────────
│││┌ @ task.jl:296 Base.unlock(%40)
││││ runtime dispatch detected: Base.unlock(%40::Any)
│││└───────────────
│┌ @ task.jl:322 Base.TaskFailedException(%68)
││ runtime dispatch detected: Base.TaskFailedException(%68::Task)
│└─

(Vector{Float64}, 6)